In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pymysql

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc #한글지원
import platform
import seaborn as sns
plt.rc("font",family='Malgun Gothic') #한글_글꼴
plt.rcParams['axes.unicode_minus'] = False #음수표시

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [11]:
df.to_csv('df_normaly.csv', index=False)
df = pd.read_csv('df_normaly.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110535 entries, 0 to 5110534
Data columns (total 26 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   time                     int64  
 1   stage                    int64  
 2   Lot                      int64  
 3   runnum                   int64  
 4   recipe                   int64  
 5   recipe_step              int64  
 6   IONGAUGEPRESSURE         float64
 7   ETCHBEAMVOLTAGE          float64
 8   ETCHBEAMCURRENT          float64
 9   ETCHSUPPRESSORVOLTAGE    float64
 10  ETCHSUPPRESSORCURRENT    float64
 11  FLOWCOOLFLOWRATE         float64
 12  FLOWCOOLPRESSURE         float64
 13  ETCHGASCHANNEL1READBACK  float64
 14  ETCHPBNGASREADBACK       float64
 15  FIXTURETILTANGLE         float64
 16  ROTATIONSPEED            float64
 17  ACTUALROTATIONANGLE      float64
 18  FIXTURESHUTTERPOSITION   float64
 19  ETCHSOURCEUSAGE          float64
 20  ETCHAUXSOURCETIMER       float64
 21  ETCHAUX2

# 모델링

## Isolation Forest
- 기존 값만으로는 fault데이터와 동일하게 정/이상을 구별해 낼 수 없음
- 연속형들의 변화는 정/이상 상태에 영향을 주지 않는 것으로 판단
- 이상 예측 모델링

In [13]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder

In [14]:
# 범주형_전처리
le = LabelEncoder()
df2 = df.copy()

result = le.fit_transform(df['max_col'])
df2['max_col'] = result
max_col_lis = le.classes_

# 모델링
df2 = df2.iloc[:, 1:-1]
model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1,
                      max_features=2, contamination=0.001)
model.fit(df2.to_numpy())

score = model.decision_function(df2.to_numpy())
anomaly = model.predict(df2.to_numpy())
df2['scores']= score
df2['anomaly']= anomaly

# 결과확인
temp = pd.concat([df2, df['target']], axis=1)
temp[temp['anomaly'] == -1]['target'].value_counts()

정상    5111
Name: target, dtype: int64

In [39]:
temp.iloc[131031:132693]

,stage,Lot,runnum,recipe,recipe_step,IONGAUGEPRESSURE,ETCHBEAMVOLTAGE,ETCHBEAMCURRENT,ETCHSUPPRESSORVOLTAGE,ETCHSUPPRESSORCURRENT,...,FIXTURESHUTTERPOSITION,ETCHSOURCEUSAGE,ETCHAUXSOURCETIMER,ETCHAUX2SOURCETIMER,ACTUALSTEPDURATION,hap,max_col,scores,anomaly,target
131031,1,9361,10548007,67,43,-0.022398,1.188755,0.789706,1.070932,0.532598,...,1.0,2.787074,3.309810,3.631607,0.803526,0.002155,9,0.101529,1,이상
131032,1,9361,10548007,67,43,-0.022398,1.189382,0.785026,1.070839,0.532254,...,1.0,2.787077,3.309812,3.631609,0.803526,0.014007,6,0.093435,1,정상
131033,1,9361,10548007,67,43,-0.019429,1.188926,0.782897,1.070839,0.532935,...,1.0,2.787080,3.309815,3.631613,0.803526,0.037351,6,0.087562,1,정상
131034,1,9361,10548007,67,43,-0.019429,1.188072,0.783535,1.070380,0.532254,...,1.0,2.787084,3.309819,3.631617,0.803526,0.002643,5,0.078066,1,정상
131035,1,9361,10548007,67,43,-0.019429,1.187048,0.784953,1.069051,0.536724,...,1.0,2.787087,3.309823,3.631621,0.803526,0.030646,6,0.094511,1,정상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132688,1,9361,10548007,67,43,-0.024606,1.189040,0.786017,1.071206,0.534658,...,1.0,2.791538,3.314805,3.636800,0.803526,0.031545,7,0.100465,1,정상
132689,1,9361,10548007,67,43,-0.024606,1.188129,0.788358,1.068820,0.543942,...,1.0,2.791542,3.314808,3.636804,0.803526,0.014932,9,0.097965,1,정상
132690,1,9361,10548007,67,43,-0.024606,1.187561,0.785664,1.070426,0.539472,...,1.0,2.791545,3.314812,3.636808,0.803526,0.023107,7,0.097189,1,정상
132691,1,9361,10548007,67,43,-0.024606,1.189494,0.788500,1.071206,0.540850,...,1.0,2.791548,3.314816,3.636812,0.803526,0.007991,4,0.083903,1,정상


In [87]:
# 범주형_전처리
le = LabelEncoder()
df2 = df.copy()

result = le.fit_transform(df['max_col'])
df2['max_col'] = result
max_col_lis = le.classes_

In [86]:
## 절대값 씌우지 않고 그냥 계산했을 때의 df로 새로 모델 돌려봄 : 결과동일
# 이상발생 시발점을 찾기 위한 df 생성
df_c = df_train.iloc[:,7:].diff()
df_c.iloc[0,:] = np.zeros(len(df_c.columns))

# 차이값 sum
df_c['hap'] = df_c.sum(axis=1)

# 가장 차가 큰 변수명
df_c['max_col'] = df_c.iloc[:,:-1].idxmax(axis=1)

# 필요한 변수만 고른 df
df_c2 = pd.concat([df_train.iloc[:, :7], df_c.iloc[:,-2:]], axis=1)
df_c2 = df_c2.drop('Tool', axis=1)

# fault
val_temp = df_fault['fault_name'].unique()
temp = df_fault[df_fault['fault_name'] == val_temp[0]][['time']]
temp['target'] = '이상'

# merge
df = pd.merge(df_c2, temp, how='outer', on='time')
df['target']= df['target'].fillna('정상')
df = df.sort_values('time')
df = df.reset_index(drop=True)

idx = df[df['stage'].isnull() == True].index

# nan 이전 시간을 이상이 발생한 시간으로 지정
for i in idx:
    df.iloc[i-1, -1] = '이상'
    
# nan값 삭제
df = df.dropna()
df = df.reset_index(drop=True)

# 변수타입
df.iloc[:,1:6] = df.iloc[:,1:6].astype('int')
df[['stage', 'Lot', 'recipe', 'recipe_step']] = df[['stage', 'Lot', 'recipe', 'recipe_step']].astype('object')
df[['max_col', 'target']] = df[['max_col', 'target']].astype('category')

df[:3]

C:\Users\USER\AppData\Local\Temp\ipykernel_4048\1731853628.py:37: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,1:6] = df.iloc[:,1:6].astype('int')


,time,stage,Lot,runnum,recipe,recipe_step,hap,max_col,target
0,3283834,1,9253,10511673,67,1,0.000000,IONGAUGEPRESSURE,정상
1,3283838,1,9253,10511673,67,1,0.009880,ETCHGASCHANNEL1READBACK,정상
2,3283842,1,9253,10511673,67,1,0.000531,FLOWCOOLFLOWRATE,정상


In [88]:
df2 = df2.iloc[:, 1:-1]
model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1,
                      max_features=2, contamination=0.001)
model.fit(df2.to_numpy())

score = model.decision_function(df2.to_numpy())
anomaly = model.predict(df2.to_numpy())
df2['scores']= score
df2['anomaly']= anomaly

,stage,Lot,runnum,recipe,recipe_step,hap,max_col,scores,anomaly
7492,147,9259,10513310,73,1,4.456790,13,-0.037852,-1
7497,147,9259,10513310,73,1,-2.638018,14,-0.022469,-1
7510,147,9259,10513310,73,1,4.729211,7,-0.026636,-1
7515,147,9259,10513310,73,1,2.606627,6,-0.016734,-1
7517,147,9259,10513310,73,1,3.132629,4,-0.015619,-1
...,...,...,...,...,...,...,...,...,...
5095323,143,764,12833961,69,13,-13.076449,12,-0.029421,-1
5097334,88,1885,12834434,68,17,-17.412402,12,-0.005647,-1
5100287,88,4329,12835151,68,13,-16.587574,12,-0.004535,-1
5103266,88,4674,12835710,68,13,-18.148654,12,-0.005428,-1


In [90]:
temp = pd.concat([df2, df['target']], axis=1)
temp[temp['anomaly'] == -1]['target'].value_counts()

정상    5111
이상       0
Name: target, dtype: int64